# Python装饰器原理

In [1]:
import time

def deco(f):
    def wrapper():
        start_time = time.time()
        f()
        end_time = time.time()
        execution_time = (end_time - start_time)*1000
        print("time is %d ms" %execution_time )
    return wrapper

@deco
def f():
    print("hello")
    time.sleep(1)
    print("world")

if __name__ == '__main__':
    f()

hello
world
time is 1000 ms


In [2]:
def makebold(fn):  
    def wrapped():  
        return "<b>" + fn() + "</b>"  
    return wrapped  
   
def makeitalic(fn):  
    def wrapped():  
        return "<i>" + fn() + "</i>"  
    return wrapped  
  
@makebold  
@makeitalic  
def hello():  
    return "hello world"  
   
print(hello()) ## 返回 <b><i>hello world</i></b> 

<b><i>hello world</i></b>


# 具体调用的过程就是将函数作为一种引用，按需求将函数放入到索要做操作的函数中去

# 以下为上课代码复现

In [3]:
original_price = [1,5,8,9,10,17,17,20,24,30,35]

In [4]:
from collections import defaultdict

In [5]:
price = defaultdict(int)

defaultdict很神奇的操作，初始化一个数据类型，可以使defaultdict(int, dict, list,str),
并且在调用不存在的索引时，不会抛出KeyError的异常

In [6]:
price =  defaultdict(int)

In [7]:
for i, p in enumerate(original_price):
    price[i + 1] = p

In [8]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35})

## 构造一个木头的单价和米数的关系

In [9]:
def example(f, arg):
    return f(arg)

In [10]:
def add_ten(num):
    return num+10

In [11]:
def mul_ten(num):
    return num*10

In [12]:
operations = [add_ten, mul_ten]
for f in operations:
    print(example(f, 100))

110
1000


In [13]:
called_time = defaultdict(int)
def get_call_time(f):
    result = f()
    print('function: {} called once!'.format(f.__name__))
    called_time[f.__name__] += 1
    return result

In [14]:
def some_function_1(): print('I am function 1')

In [15]:
get_call_time(some_function_1)

I am function 1
function: some_function_1 called once!


In [16]:
called_time

defaultdict(int, {'some_function_1': 1})

In [17]:
call_time_with_arg = defaultdict(int)

In [18]:
def r(n):
    #fname = r.__name__
    fname = r.__name__
    
    call_time_with_arg[(fname, n )] += 1
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)]
    )

In [19]:
r(15)

45

In [20]:
call_time_with_arg

defaultdict(int,
            {('r', 1): 3188646,
             ('r', 2): 1062882,
             ('r', 3): 354294,
             ('r', 4): 118098,
             ('r', 5): 39366,
             ('r', 6): 13122,
             ('r', 7): 4374,
             ('r', 8): 1458,
             ('r', 9): 486,
             ('r', 10): 162,
             ('r', 11): 54,
             ('r', 12): 18,
             ('r', 13): 6,
             ('r', 14): 2,
             ('r', 15): 1})

In [21]:
from collections import Counter

In [22]:
Counter(call_time_with_arg).most_common()

[(('r', 1), 3188646),
 (('r', 2), 1062882),
 (('r', 3), 354294),
 (('r', 4), 118098),
 (('r', 5), 39366),
 (('r', 6), 13122),
 (('r', 7), 4374),
 (('r', 8), 1458),
 (('r', 9), 486),
 (('r', 10), 162),
 (('r', 11), 54),
 (('r', 12), 18),
 (('r', 13), 6),
 (('r', 14), 2),
 (('r', 15), 1)]

In [23]:
def p(n):
    return [n] + [p(i) + p(n-i) for i in range(1,n)]

In [24]:
p(4)

[4,
 [1, 3, [1, 2, [1, 1]], [2, [1, 1], 1]],
 [2, [1, 1], 2, [1, 1]],
 [3, [1, 2, [1, 1]], [2, [1, 1], 1], 1]]

In [25]:
called_time_with_arg = defaultdict(int)
def get_call_time(f):
    '''@param f is a function'''
    def wrap(n):
        result = f(n)
        call_time_with_arg[(f.__name__, n)] += 1
        return result
    return wrap
        

In [26]:
def add_ten(n): return n+10

In [27]:
add_ten_with_call_time = get_call_time(add_ten)

In [28]:
add_ten_with_call_time

<function __main__.get_call_time.<locals>.wrap>

In [29]:
r(4)

10

# 上述其实本质上可以理解为查找树
            4                4
           / \             /  \
          2   2           1    3
         / \ / \          |   /  \
        1  1 1  1         1   1   2
                                 / \
                                 1  1

#### 进入正题

首先要实现的是每次计算，都要反馈一个计算的是多长的钢筋长度

最后的结果要显示出{(i, n-i), count}

In [30]:
from collections import Counter

In [31]:
from collections import defaultdict

In [32]:
already_computed = defaultdict(list)

In [33]:
from functools import wraps

In [34]:
@get_call_time
def r(n):
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1, n)]
    )

In [35]:
r

<function __main__.get_call_time.<locals>.wrap>

In [36]:
help(r)

Help on function wrap in module __main__:

wrap(n)



In [37]:
r(15)

45

In [38]:
#### help在不引入wraps的时候，会显示的是wrap的信息

In [39]:
def memo(f):
    already_computed  = {}
    def _wrap(arg):
        result = None
        if arg in already_computed: 
            result = already_computed[arg]
        else:
            result = f(arg)
            already_computed[arg] = result
        return result
    return _wrap

In [40]:
@get_call_time
@memo
def r(n):
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1, n)]
    )

In [41]:
r(19)

57

In [42]:
call_time_with_arg

defaultdict(int,
            {('_wrap', 1): 36,
             ('_wrap', 2): 34,
             ('_wrap', 3): 32,
             ('_wrap', 4): 30,
             ('_wrap', 5): 28,
             ('_wrap', 6): 26,
             ('_wrap', 7): 24,
             ('_wrap', 8): 22,
             ('_wrap', 9): 20,
             ('_wrap', 10): 18,
             ('_wrap', 11): 16,
             ('_wrap', 12): 14,
             ('_wrap', 13): 12,
             ('_wrap', 14): 10,
             ('_wrap', 15): 8,
             ('_wrap', 16): 6,
             ('_wrap', 17): 4,
             ('_wrap', 18): 2,
             ('_wrap', 19): 1,
             ('r', 1): 6377310,
             ('r', 2): 2125770,
             ('r', 3): 708590,
             ('r', 4): 236197,
             ('r', 5): 78732,
             ('r', 6): 26244,
             ('r', 7): 8748,
             ('r', 8): 2916,
             ('r', 9): 972,
             ('r', 10): 324,
             ('r', 11): 108,
             ('r', 12): 36,
             ('r', 13): 12,
   

#### 怎么切分

In [50]:
solution = {}

In [51]:
@memo
def r(n):
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )
    solution[n] = (n - max_split, max_split)
    return max_price

In [52]:
already_computed ={}

In [53]:
r(10)

30

In [54]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0)}

In [55]:
r(100)

316

In [56]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3),
 21: (11, 10),
 22: (11, 11),
 23: (22, 1),
 24: (22, 2),
 25: (22, 3),
 26: (24, 2),
 27: (25, 2),
 28: (22, 6),
 29: (28, 1),
 30: (28, 2),
 31: (28, 3),
 32: (22, 10),
 33: (22, 11),
 34: (33, 1),
 35: (33, 2),
 36: (33, 3),
 37: (35, 2),
 38: (36, 2),
 39: (33, 6),
 40: (39, 1),
 41: (39, 2),
 42: (39, 3),
 43: (33, 10),
 44: (33, 11),
 45: (44, 1),
 46: (44, 2),
 47: (44, 3),
 48: (46, 2),
 49: (47, 2),
 50: (44, 6),
 51: (50, 1),
 52: (50, 2),
 53: (50, 3),
 54: (44, 10),
 55: (44, 11),
 56: (55, 1),
 57: (55, 2),
 58: (55, 3),
 59: (57, 2),
 60: (58, 2),
 61: (55, 6),
 62: (61, 1),
 63: (61, 2),
 64: (61, 3),
 65: (55, 10),
 66: (55, 11),
 67: (66, 1),
 68: (66, 2),
 69: (66, 3),
 70: (68, 2),
 71: (69, 2),
 72: (66, 6),
